# 動画内の人物・車を検出してモザイク処理するプログラム

# Googleドライブへのマウント

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# pipインストール

In [2]:
os.chdir("/content/drive/MyDrive/Python_Colab/papu_git/video_mosaic_person_car/")
!pip install -r requirements.txt
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: opencv-python--headless
    Found existing installation: opencv-python-headl

# ライブラリインポート

In [4]:
import torch
import time
import os
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
from matplotlib import pyplot as plt
import shutil

# 必要な画像・動画処理の関数リスト
ーーーーーーーーーーーーーーーーーーーーーーーー
\
静止画像の処理（2値化・モザイク化・輝度化など）
\
ーーーーーーーーーーーーーーーーーーーーーーーー

In [3]:
# 画像処理の関数をまとめたファイル

def brightness(input_image , gamma):
  """ 
  画像の明るさを変える関数
  gamma > 1  =>  明るくなる
  gamma < 1  =>  暗くなる 
  """
  img2gamma = np.zeros((256,1),dtype=np.uint8)  # ガンマ変換初期値

  for i in range(256):
    # ガンマ補正の公式 : Y = 255(X/255)**(1/γ)
    img2gamma[i][0] = 255 * (float(i)/255) ** (1.0 /gamma)
  
  # 読込画像をガンマ変換
  gamma_img = cv2.LUT(img , img2gamma)
  return gamma_img


def mosaic(input_image , parameter):
  """ 
  モザイク処理（画像を縮小してモザイク効果を適用）
  parameter : リサイズにおける 縦＝横 サイズ（小さいほどモザイクが強くなる）
  """
  # 画像のサイズを取得
  height, width = input_image.shape[:2]

  # モザイク処理
  mosaic_img = cv2.resize(input_image, (parameter , parameter), interpolation=cv2.INTER_NEAREST)
  mosaic_img = cv2.resize(mosaic_img, input_image.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)

  return mosaic_img


def mask(input_image , threshold):
  """ 
   2値化（マスク）処理 
   threshold : しきい値（ 0 ~ 255 の 整数値）
  """
  # 元のRGB画像をコピーする
  outpu_img = input_image.copy()

  # グレースケール化
  img_gray = cv2.cvtColor(input_image, cv2.COLOR_RGB2GRAY)
  # 2値化
  ret, mask_img = cv2.threshold(img_gray, threshold, 255, cv2.THRESH_BINARY)
  # 2値画像を3チャンネルに拡張する
  mask_img_3ch = cv2.cvtColor(mask_img, cv2.COLOR_GRAY2RGB)

  return mask_img_3ch


def detection_area_conversion(
    input_img_path , 
    output_img_path , 
    label_information , 
    model ,
):
  """
  特定の検出物体に画像処理 を実行する関数
  label_information : 「検出対象の種類・画像処理方法・パラメータ」を numpyで格納
  model : 物体検出のAIモデル
  """

  def yolov5_to_np(obj):
    """ 検出結果をNumpyに変換する関数 """
    obj_array = np.empty( ( len(obj) + 1 ,  len(obj.columns) ), dtype = 'object' )
    obj_array[0 , :] = obj.columns
    obj_array[1: , :] = obj.values
    return obj_array

  def select_detection_area(detection_results , label_information):
    """検出領域を選択し画像処理を実行 """

    obj = detection_results.pandas().xyxy[0]
    obj_array = yolov5_to_np(obj)

    for n in range( len(obj) ):
      # 検出領域（短形ボックス）の座標を取得
      ymin  , ymax = obj_array[n+1 , 1]  ,  obj_array[n+1 , 3]
      xmin , xmax =  obj_array[n+1 , 0]  ,  obj_array[n+1 , 2]

      # 検出領域の種類（ラベル）に応じて、任意の画像処理とパラメータを実行
      for label_number in range( label_information.shape[0] ):
        
        # 例）ラベル番号 ＝ 0  =>  carラベル , モザイク処理 , 0.5 の場合 など
        # 複数ある検出領域の中から「carラベル」を探す
        if obj_array[n+1 , 6] == label_information[label_number][0] :
          parameter = label_information[label_number][2]

          # 短形ボックスのみに画像処理を実行
          output_image[ int(ymin) : int(ymax)  ,  int(xmin) : int(xmax) ]   \
          = label_information[label_number][1](
              input_image[ int(ymin) : int(ymax)  ,  int(xmin) : int(xmax) ]  , 
              parameter
          )
    return output_image
  
  # 入力画像の準備
  input_image = cv2.imread(input_img_path)
  input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)  # RGB化
  output_image = input_image.copy()

  # 物体検出を実行し、結果を取得
  detection_results = model(input_img_path)

  # 任意の物体の検出領域に任意の法則で画像処理を実行
  output_image = select_detection_area(
      detection_results , label_information
  )
  
  # 出力画像を保存
  output_image = cv2.cvtColor(output_image , cv2.COLOR_RGB2BGR)  # BGR化
  cv2.imwrite( output_img_path , output_image )


def video_information(input_video_file , output_video_file):
  """
  総再生時間、総フレーム数の表示 : frame_number , total_time
  動画の書き込み形式の取得 ： fmt , writer
  """

  # 動画をフレームに分割
  cap = cv2.VideoCapture(input_video_file)

  #動画サイズ取得
  width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH) )
  height = int( cap.get(cv2.CAP_PROP_FRAME_HEIGHT) )

  #フレームレート取得
  fps = cap.get(cv2.CAP_PROP_FPS)

  #フォーマット指定（動画の書き込み形式）
  fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
  writer = cv2.VideoWriter( output_video_file , fmt, fps, (width, height) )

  # 表示
  print()
  print("合計フレーム数：")
  frame_number = int( cap.get(cv2.CAP_PROP_FRAME_COUNT) )
  print(f"{frame_number} 枚 \n")
  print("合計再生時間（総フレーム数 / FPS）：")
  total_time = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
  total_time = round(total_time , 3)
  print(f"{total_time} 秒 \n")
  print()

  return cap , writer , fmt , fps , width , height

# 学習済みYolov5モデルの読み込み

In [ ]:
# ローカルディレクトリから読み込み（yolov5 をクローン済みの場合 & モデルを読み込み２回目以降）
os.chdir("/content/drive/MyDrive/Python_Colab/papu_git/video_mosaic_person_car/yolov5/")
model = torch.load("../yolov5s.pth")

In [ ]:
# 初回のみ（yolov5 をクローンしていない場合）
os.chdir("/content/drive/MyDrive/Python_Colab/papu_git/video_mosaic_person_car/")
!git clone https://github.com/ultralytics/yolov5

#「yolov5」ディレクトリに移動
os.chdir("/content/drive/MyDrive/Python_Colab/papu_git/video_mosaic_person_car/yolov5/")

#「yolov5」モデルを読み込み（初回の場合）
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# モデルの保存は本ファイルと同じディレクトリ
torch.save(model , "../yolov5s.pth")

# 動画の人物・車のモザイク処理を実行

In [6]:
def video_convert(
    current_dir ,
    input_video_file ,
    input_frame_folda ,
    output_frame_folda ,
    output_video_file ,
    fps_divide ,
    model ,
    detection_area_conversion ,
    label_information ,
  ):
  """
  動画に物体検出を実行する関数

  video_input_file : 入力動画のファイル名
  video_output_folda : 動画の出力先のフォルダ名
  fps_divide : フレーム数の縮小比 ＝ 処理速度の倍率
  model : Yolo5 物体検出のAIモデル

  label_information : 「検出対象の種類 , 画像処理方法 , パラメータ」numpy配列
  ① 検出対象の種類：「文字列（pytorchのYolo5のラベル）」を引数
  ② 画像処理方法：「画像処理を実行する関数」を引数
  ③ パラメータ：「数字」を引数
  """

  def frame_detect(
    current_dir , 
    input_frame_file , 
    output_frame_folda ,
    model ,
  ):
    """
    動画のフレーム画像に物体検出を実行する関数
    ※ 検出結果のフレーム画像は毎回フォルダごと削除する必要がある
    """
    # フレーム画像の物体検出
    frame_detection = model(input_frame_file)

    # 一つ前のフレームの検出結果をフォルダごと削除
    if os.path.exists(output_frame_folda) == True:
      shutil.rmtree(output_frame_folda)

    # 今回のフレームの検出結果を保存
    frame_detection.save(save_dir = output_frame_folda)
  #================================================

  # 入力動画をフレームに分割、動画出力の形式・総再生時間などを取得・表示
  cap , writer , fmt , fps , width , height \
  = video_information(input_video_file , output_video_file)

  count = 1
  while True :
      print("Frame number: "+ str(count))

      #フレーム情報取得
      ret, img = cap.read()
      
      #動画が終われば処理終了
      if ret == False:
          break
      
      # 静止フレーム画像の物体検出を実行
      if ( count % fps_divide == 0 ) :

        # フレーム画像のファイル名を決定
        frame_file = "frame" + str(count) + ".jpg"
        input_frame_file = input_frame_folda + frame_file
        output_frame_file = output_frame_folda + frame_file

        # 入力フレーム画像をフォルダに保存
        cv2.imwrite(input_frame_file , img)

        # 入力フレーム画像に物体検出を実行
        frame_detect(current_dir , input_frame_file , output_frame_folda , model)

        # 入力フレーム画像内で検出領域の一部に画像処理
        detection_area_conversion(
            input_frame_file , output_frame_file , label_information , model
        )
        #====================================

        # 今回の入力フレームを削除（フォルダごと削除して新しくフォルダを作り直す）
        if os.path.exists(input_frame_folda) == True:
          shutil.rmtree(input_frame_folda)
          os.mkdir(input_frame_folda)
        
          # 出力フレーム画像を動画の一部として書き込み
          frame_detection = cv2.imread(output_frame_file)
          writer.write(frame_detection)

      if ( count < fps_divide ):
        # 最初の(fps_divide + 1)回だけ入力フレームをそのまま書き込み
        writer.write(img)
      
      if ( count % fps_divide != 0  and  count >= fps_divide + 1 ) :
        # 間引いたフレームは検出が行われたフレームの出力を引き継ぐ
        writer.write(frame_detection)
      
      count += 1
      print()

  # ループ終了

  # フレームの検出結果をフォルダごと削除
  shutil.rmtree(output_frame_folda)
  
  cap.release()
  #これを忘れるとプログラムが出力ファイルを開きっぱなしになる
  writer.release()
  cv2.destroyAllWindows()



In [7]:
if __name__ == '__main__':
  # 現在のディレクトリに移動
  current_dir = "/content/drive/MyDrive/Python_Colab/papu_git/video_mosaic_person_car/"
  os.chdir(current_dir)

  # 入力動画と出力先を定義
  input_video_file = "./detect_data/input_video/" + "soccer.mp4"
  input_frame_folda = "./detect_data/input_frame/"
  output_frame_folda = "./detect_data/output_frame/"
  output_video_file = "./detect_data/output_video/" + "soccer_mosaic2.mp4"

  # 動画におけるフレーム数の縮小比 ＝ 処理速度の倍率
  fps_divide = 8

  # 検出対象の種類・画像処理方法（関数を引数）・パラメータを格納
  label_information = np.array([
      [ 'car' , mosaic , 25 ] , 
      [ 'person' , mosaic , 25 ]
  ])
  # 車・人物を検出してモザイク

  video_convert(
      current_dir ,
      input_video_file ,
      input_frame_folda ,
      output_frame_folda ,
      output_video_file ,
      fps_divide ,
      model ,
      detection_area_conversion ,  # 検出物体の一部に画像処理を実行する関数
      label_information , # 検出物体の種類と画像処理の内容・パラメータ情報
  )


合計フレーム数：
7890 枚 

合計再生時間（総フレーム数 / FPS）：
131.5 秒 


Frame number: 1

Frame number: 2

Frame number: 3

Frame number: 4

Frame number: 5

Frame number: 6

Frame number: 7

Frame number: 8


Saved 1 image to detect_data/output_frame



Frame number: 9

Frame number: 10

Frame number: 11

Frame number: 12

Frame number: 13

Frame number: 14

Frame number: 15

Frame number: 16


Saved 1 image to detect_data/output_frame



Frame number: 17

Frame number: 18

Frame number: 19

Frame number: 20

Frame number: 21

Frame number: 22

Frame number: 23

Frame number: 24


...


Frame number: 7873

Frame number: 7874

Frame number: 7875

Frame number: 7876

Frame number: 7877

Frame number: 7878

Frame number: 7879

Frame number: 7880


Saved 1 image to detect_data/output_frame



Frame number: 7881

Frame number: 7882

Frame number: 7883

Frame number: 7884

Frame number: 7885

Frame number: 7886

Frame number: 7887

Frame number: 7888


Saved 1 image to detect_data/output_frame



Frame number: 7889

Frame number: 7890

Frame number: 7891
